guide taken from: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?tabs=command-line&pivots=programming-language-python

In [1]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 70.3/70.3 kB 3.8 MB/s eta 0:00:00
     -------------------------------------- 319.8/319.8 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 56.1/56.1 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 61.2/61.2 kB 3.2 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import os
import requests
import json
import openai

In [2]:
openai.api_key = "404a44ab43b741f99e1ee85bffa5516b"
openai.api_base =  "https://jcopenaitest041223.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2022-12-01' # this may change in the future

deployment_name='genText' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

In [3]:
# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'Write a tagline for a surf shop. '
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text)

Sending a test completion job


APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='jcopenaitest041223.openai.azure.com', port=443): Max retries exceeded with url: //openai/deployments/genText/completions?api-version=2022-12-01 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)')))

## Code without SSL Verification

In [5]:
import contextlib
import warnings
from urllib3.exceptions import InsecureRequestWarning

# Use a context manager for disabling SSL verification with 3rd party modules
old_merge_environment_settings = requests.Session.merge_environment_settings

@contextlib.contextmanager
def no_ssl_verification():
    opened_adapters = set()

    def merge_environment_settings(self, url, proxies, stream, verify, cert):
        # Verification happens only once per connection so we need to close
        # all the opened adapters once we're done. Otherwise, the effects of
        # verify=False persist beyond the end of this context manager.
        opened_adapters.add(self.get_adapter(url))

        settings = old_merge_environment_settings(
            self, url, proxies, stream, verify, cert)
        settings['verify'] = False

        return settings

    requests.Session.merge_environment_settings = merge_environment_settings

    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', InsecureRequestWarning)
            yield
    finally:
        requests.Session.merge_environment_settings = old_merge_environment_settings

        for adapter in opened_adapters:
            try:
                adapter.close()
            except:
                pass

# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'Write a tagline for an ice cream shop. '
with no_ssl_verification():
    response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=20)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text)

Sending a test completion job
Write a tagline for an ice cream shop. 23 hours leftClassic C...Creamery is a local, family-owned company with three locations within
